# I/O

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#I/O" data-toc-modified-id="I/O-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>I/O</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Reading-and-Writing-Files" data-toc-modified-id="Reading-and-Writing-Files-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Reading and Writing Files</a></span></li><li><span><a href="#Opening-xarray-datasets" data-toc-modified-id="Opening-xarray-datasets-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Opening xarray datasets</a></span></li><li><span><a href="#Saving-xarray-datasets-as-netcdf-files" data-toc-modified-id="Saving-xarray-datasets-as-netcdf-files-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Saving xarray datasets as netcdf files</a></span></li><li><span><a href="#Multifile-datasets" data-toc-modified-id="Multifile-datasets-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Multifile datasets</a></span></li><li><span><a href="#Zarr" data-toc-modified-id="Zarr-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Zarr</a></span></li><li><span><a href="#Going-Further" data-toc-modified-id="Going-Further-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Going Further</a></span></li></ul></li></ul></div>

## Learning Objectives

- Write xarray objects to netCDF files
- Load xarray datasets from netCDF files
- Provide a brief overview on Zarr

## Reading and Writing Files


Xarray supports direct serialization and I/O to several file formats including pickle, netCDF, OPeNDAP (read-only), GRIB1/2 (read-only), Zarr, and HDF by integrating with third-party libraries. Additional serialization formats for 1-dimensional data are available through pandas.

File types
- Pickle
- NetCDF 3/4
- RasterIO
- Zarr
- PyNio

Interoperability
- Pandas
- Iris
- CDMS
- dask DataFrame


## Opening xarray datasets

Xarray's `open_dataset` and `open_mfdataset` are the primary functions for opening local or remote datasets such as netCDF, GRIB, OpenDap, and HDF. These operations are all supported by third party libraries (engines) for which xarray provides a common interface. 

In [1]:
!ncdump -h ./data/rasm.nc

netcdf rasm {
dimensions:
	time = 36 ;
	y = 205 ;
	x = 275 ;
variables:
	double Tair(time, y, x) ;
		Tair:_FillValue = 9.96920996838687e+36 ;
		Tair:units = "C" ;
		Tair:long_name = "Surface air temperature" ;
		Tair:type_preferred = "double" ;
		Tair:time_rep = "instantaneous" ;
		Tair:coordinates = "yc xc" ;
	double time(time) ;
		time:_FillValue = NaN ;
		time:long_name = "time" ;
		time:type_preferred = "int" ;
		time:units = "days since 0001-01-01" ;
		time:calendar = "noleap" ;
	double xc(y, x) ;
		xc:_FillValue = NaN ;
		xc:long_name = "longitude of grid cell center" ;
		xc:units = "degrees_east" ;
		xc:bounds = "xv" ;
	double yc(y, x) ;
		yc:_FillValue = NaN ;
		yc:long_name = "latitude of grid cell center" ;
		yc:units = "degrees_north" ;
		yc:bounds = "yv" ;

// global attributes:
		:title = "/workspace/jhamman/processed/R1002RBRxaaa01a/lnd/temp/R1002RBRxaaa01a.vic.ha.1979-09-01.nc" ;
		:institution = "U.W." ;
		:source = "RACM R1002RBRxaaa01a" ;
		:output_frequency = "daily"

In [2]:
import xarray as xr
from glob import glob

In [3]:
ds = xr.open_dataset('./data/rasm.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 36, x: 275, y: 205)
Coordinates:
  * time     (time) object 1980-09-16 12:00:00 ... 1983-08-17 00:00:00
    xc       (y, x) float64 ...
    yc       (y, x) float64 ...
Dimensions without coordinates: x, y
Data variables:
    Tair     (time, y, x) float64 ...
Attributes:
    title:                     /workspace/jhamman/processed/R1002RBRxaaa01a/l...
    institution:               U.W.
    source:                    RACM R1002RBRxaaa01a
    output_frequency:          daily
    output_mode:               averaged
    convention:                CF-1.4
    references:                Based on the initial model of Liang et al., 19...
    comment:                   Output from the Variable Infiltration Capacity...
    nco_openmp_thread_number:  1
    NCO:                       "4.6.0"
    history:                   Tue Dec 27 14:15:22 2016: ncatted -a dimension...

## Saving xarray datasets as netcdf files

Xarray provides a high-level method for writing netCDF files directly from Xarray Datasets/DataArrays.

In [4]:
ds.to_netcdf('./data/rasm_test.nc')

## Multifile datasets

Xarray can read/write multifile datasets using the `open_mfdataset` and `save_mfdataset` functions. 

In [5]:
paths = glob('./data/19*.nc')
paths

['./data/1980.nc', './data/1981.nc', './data/1982.nc', './data/1983.nc']

In [6]:
ds2 = xr.open_mfdataset(paths)
ds2

/Users/abanihi/opt/miniconda3/envs/analysis/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/Users/abanihi/opt/miniconda3/envs/analysis/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset` to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in in future, please use the
new `combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  coords=coords)


<xarray.Dataset>
Dimensions:  (time: 36, x: 275, y: 205)
Coordinates:
    xc       (y, x) float64 189.2 189.4 189.6 189.7 ... 17.65 17.4 17.15 16.91
    yc       (y, x) float64 16.53 16.78 17.02 17.27 ... 28.26 28.01 27.76 27.51
  * time     (time) object 1980-09-16 12:00:00 ... 1983-08-17 00:00:00
Dimensions without coordinates: x, y
Data variables:
    Tair     (time, y, x) float64 dask.array<shape=(36, 205, 275), chunksize=(4, 205, 275)>
Attributes:
    title:                     /workspace/jhamman/processed/R1002RBRxaaa01a/l...
    institution:               U.W.
    source:                    RACM R1002RBRxaaa01a
    output_frequency:          daily
    output_mode:               averaged
    convention:                CF-1.4
    references:                Based on the initial model of Liang et al., 19...
    comment:                   Output from the Variable Infiltration Capacity...
    nco_openmp_thread_number:  1
    NCO:                       "4.6.0"
    history:             

## Zarr

Zarr is a Python package providing an implementation of chunked, compressed, N-dimensional arrays. Zarr has the ability to store arrays in a range of ways, including in memory, in files, and in cloud-based object storage such as Amazon S3 and Google Cloud Storage. Xarray’s Zarr backend allows xarray to leverage these capabilities.

In [7]:
# save to a Zarr dataset
ds.to_zarr('./data/rasm.zarr', mode='w')

In [8]:
!ls ./data/rasm.zarr

Tair time xc   yc


In [9]:
!du -h ./data/rasm.zarr

348K	./data/rasm.zarr/yc
 12K	./data/rasm.zarr/time
7.6M	./data/rasm.zarr/Tair
332K	./data/rasm.zarr/xc
8.3M	./data/rasm.zarr


## Going Further
    
- Xarray I/O Documentation: http://xarray.pydata.org/en/latest/io.html

- Zarr Documentation: https://zarr.readthedocs.io/en/stable/



<div class="alert alert-block alert-success">
  <p>Previous: <a href="01_getting_started_with_xarray.ipynb">Getting Started with Xarray</a></p>
  <p>Next: <a href="03_indexing.ipynb">Indexing</a></p>
</div>